In this notebook we analyze the charactristics of the generated item and user groups.

In [1]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
from tqdm import tqdm

In [2]:
ds_name = "Gowalla"

## Item Groups

In [3]:
item_top_fraction = 0.2

In [4]:
# loading ds_name dataset
dataset = pd.read_csv(f"datasets/{ds_name}/{ds_name}_train.txt", sep="\t", names=['uid', 'iid', 'count'])

In [5]:
# To read the popularit of itmes from a implicit feedback type datasets
def read_item_popularity():
    print("Reading implicit item popularity ...")
    items_freq = dict()
    global user_interactions
    for eachline in tqdm(dataset.itertuples(index=True)):
        # iid, count = int(eachline.iid), int(eachline.count)
        uid, iid, count = int(eachline.uid), int(eachline.iid), int(eachline.count)
        uid, iid, count = int(uid), int(iid), int(count)
        # Adding users and their corresponding items
        # We need to loop on the user interaction frequency with item in implicit feedback
        for i in range(count):
            if uid in user_interactions.keys():
                user_interactions[uid].append(iid)
            else:
                user_interactions[uid] = [iid]
        # Adding the interaction frequency (implicit) of items
        if iid in items_freq.keys():
            items_freq[iid] += count
        else:
            items_freq[iid] = count
    return items_freq

In [12]:
# len(user_interactions[188])

In [7]:
# A dictionary to store users and the items in their profiles 
user_interactions = defaultdict(list)

In [8]:
item_freq = read_item_popularity()
num_items = len(item_freq)
print('No. items: ' + str(num_items))

24272it [00:00, 242683.20it/s]

Reading implicit item popularity ...


262714it [00:01, 262133.27it/s]

No. items: 30943


In [8]:
# get top items
num_top = int(item_top_fraction * num_items)
# sort items according to the no. of recevied interactions from users
sorted_item_freq = {k: v for k, v in sorted(item_freq.items(), key=lambda item: item[1], reverse=True)}
short_heads = list(sorted_item_freq.keys())[:num_top] # top popular items based on the item_top_fractin
long_tails = list(sorted_item_freq.keys())[num_top:] # log-tail items
print('No. top items: ' + str(num_top))

No. top items: 3115


In [9]:
# read check-in frequency from train data
train_data = open(f"datasets/{ds_name}/{ds_name}_train.txt", 'r').readlines()
# uid -> set([lids])
uid_to_lids = defaultdict(set)
# uid -> check-in freq
uid_to_freq = dict()

for eachline in tqdm(train_data):
    uid, lid, freq = eachline.strip().split()
    uid, lid, freq = int(uid), int(lid), int(freq)
    if uid in uid_to_freq.keys():
        uid_to_lids[uid].add(lid)
        uid_to_freq[uid] += freq
    else:
        uid_to_lids[uid] = {lid}
        uid_to_freq[uid] = freq

100%|██████████| 196866/196866 [00:00<00:00, 468085.46it/s]


In [11]:
leisure_users = pd.read_csv(f"groups/user_groups/{ds_name}/leisure.txt", names=['uid'])
working_users = pd.read_csv(f"groups/user_groups/{ds_name}/working.txt", names=['uid'])

In [12]:
def pop_item_counter(uid):
    global uid_to_lids
    pop_in_profile_count = 0
    for eachuid in uid_to_lids[uid]:
        if int(eachuid) in set(short_heads):
            pop_in_profile_count += 1
    return pop_in_profile_count

In [13]:
leisure_pop_profile_count = 0
working_pop_profile_count = 0

for eachuid in tqdm(leisure_users.itertuples(index=True)):
    leisure_pop_profile_count += len(uid_to_lids[eachuid.uid] & set(short_heads)) / len(uid_to_lids[eachuid.uid])
    # leisure_pop_profile_count += pop_item_counter(uid=eachuid.uid)

for eachuid in tqdm(working_users.itertuples(index=True)):
    working_pop_profile_count += len(uid_to_lids[eachuid.uid] & set(short_heads)) / len(uid_to_lids[eachuid.uid])
    # working_pop_profile_count += pop_item_counter(uid=eachuid.uid)

print(f"Avg. Pop Items in Leisure Users: {leisure_pop_profile_count / leisure_users.shape[0]}")
print(f"Avg. Pop Items in Working Users: {working_pop_profile_count / working_users.shape[0]}")

1427it [00:00, 7329.06it/s]
1427it [00:00, 7884.80it/s]

Avg. Pop Items in Leisure Users: 0.6145489810279476
Avg. Pop Items in Working Users: 0.6581704268634869


In [14]:
leisure_users_profile_size = 0
working_users_profile_size = 0

for eachuid in leisure_users.itertuples(index=True):
    leisure_users_profile_size += uid_to_freq[eachuid.uid]

for eachuid in working_users.itertuples(index=True):
    working_users_profile_size += uid_to_freq[eachuid.uid]

print(f"Avg. Leisure Profile Size: {leisure_users_profile_size / leisure_users.shape[0]}")
print(f"Avg. Working Profile Size: {working_users_profile_size / working_users.shape[0]}")

Avg. Leisure Profile Size: 313.1030133146461
Avg. Working Profile Size: 31.384022424667133


In [15]:
leisure_users_profile_size

446798

In [16]:
working_users_profile_size

44785